In [77]:
import sys
sys.path.append("../../models/PFN")
from dataset_pfn import PFNDataset
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from pfn_model import ParticleFlowNetwork as Model
import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
import os, json
from scipy import interpolate
%env HDF5_USE_FILE_LOCKING=FALSE

env: HDF5_USE_FILE_LOCKING=FALSE


In [78]:
all_models = [f for f in os.listdir("../../models/PFN/trained_models") if "_best" in f and "_aug" not in f]
print("\n".join(all_models))

PFN_best_latent_128
PFN_best_modified_v7
PFN_best_modified_v4
PFN_best_v9
PFN_best
PFN_best_latent_64


In [79]:
test_path = "../../datasets/test.h5"

#Loading testing dataset
test_set = PFNDataset(test_path, preprocessed=True)
testloader = DataLoader(test_set, shuffle=True, batch_size=500, num_workers=30, pin_memory=True, persistent_workers=True)

/u/avroy/.conda/envs/toptagger_env/lib/python3.9/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 30 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [80]:
def eval2(model):
    labels = []
    preds = []
    with torch.no_grad():
        for x,m,y,_ in tqdm(testloader):
            x = x.cuda()
            m = m.cuda()
            pred = model(x, m)
            labels.append(y[:,0].cpu().numpy())
            preds.append(pred[:,0].cpu().numpy())
    labels = np.concatenate(labels, axis=None)
    preds = np.concatenate(preds, axis=None)
    return labels, preds

In [81]:
#loading model
for modelname in all_models:
    model_dict = json.load(open("../../models/PFN/trained_model_dicts/" + modelname.replace("_best","") + ".json"))
    #print(modelname, model_dict)
    label = model_dict['label']
    f_nodes = list(map(int, model_dict['f_nodes'].split(',')))
    phi_nodes = list(map(int, model_dict['phi_nodes'].split(',')))

    model = Model(input_dims=3, Phi_sizes=phi_nodes, F_sizes=f_nodes).cuda()
    model.load_state_dict(torch.load("../../models/PFN/trained_models/" + modelname ))
    nparams = sum(p.numel() for p in model.parameters())
    
    FcLayers = []
    PhiLayers = []
    for layer in model._modules['fc']:
        #print(layer)
        FcLayers.append(layer)
    #print('\n')
    for layer in model._modules['phi']:
        #print(layer)
        PhiLayers.append(layer)
    
    
    labels, preds = eval2(model)
    accuracy = accuracy_score(labels, preds.round())*100
    
    auc = roc_auc_score(labels, preds)*100
    
    fpr, tpr, thresholds = roc_curve(labels, preds, drop_intermediate=False)
    intrp = interpolate.interp1d(fpr, tpr)
    brr = 1./(1- intrp(0.7))
    print("{} \t\t Params: {}\t ROC-AUC: {:.2f}%, Accuracy: {:.2f}%, BRR: {:.2f}".format(modelname,nparams,auc,accuracy,brr))
    #print(modelname, "\t", "ROC-AUC: {:.4f}% Accuracy: {:.4f}%".format(auc, accuracy))

[100, 100, 128] [100, 100, 100]


100%|██████████| 808/808 [00:05<00:00, 157.08it/s]


PFN_best_latent_128 		 Params: 56730	 ROC-AUC: 99.74%, Accuracy: 97.58%, BRR: 33652.33
[100, 64, 64] [64, 32]


100%|██████████| 808/808 [00:03<00:00, 269.24it/s]


PFN_best_modified_v7 		 Params: 17330	 ROC-AUC: 99.45%, Accuracy: 96.89%, BRR: 1373.56
[100, 64, 64] [32, 32]


...

PFN_best_v9 		 Params: 25414	 ROC-AUC: 99.50%, Accuracy: 97.19%, BRR: 2060.35
[100, 100, 256] [100, 100, 100]


100%|██████████| 808/808 [00:03<00:00, 210.44it/s]


PFN_best 		 Params: 82458	 ROC-AUC: 99.74%, Accuracy: 97.72%, BRR: 8778.87
[100, 100, 64] [100, 100, 100]


100%|██████████| 808/808 [00:03<00:00, 251.74it/s]


PFN_best_latent_64 		 Params: 43866	 ROC-AUC: 99.48%, Accuracy: 97.10%, BRR: 1696.76
